<a href="https://colab.research.google.com/github/JCuomo/CameraTraps/blob/main/Separate_detections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1 Set up the Colab instance to run on a GPU accelerator


Navigate to Edit→Notebook Settings and select "GPU" from the "Hardware accelerator" drop-down menu.

## 2 Mount Google Drive in Colab

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# 3 Configurable Parameters

In [2]:
# Choose where the images are
images_dir = '/content/drive/MyDrive/Camaras Aspen/sample'
# Choose a location for the output JSON file
output_file_path = '/content/drive/MyDrive/Camaras Aspen/sample/sample.json'
# Select a threshold for the detection
confidence_threshold = 0.2

## 4 Run all
Run all below cells (Ctrl+F10)


In [3]:
pip install humanfriendly jsonpickle ultralytics

### Download the MegaDetector model files

In [4]:
!wget -O /content/md_v5a.0.0.pt https://lila.science/public/md_rebuild/md_v5a.0.0_rebuild_pt-1.12_zerolr.pt
!wget -O /content/md_v5b.0.0.pt https://lila.science/public/md_rebuild/md_v5b.0.0_rebuild_pt-1.12_zerolr.pt

--2023-07-11 01:17:49--  https://lila.science/public/md_rebuild/md_v5a.0.0_rebuild_pt-1.12_zerolr.pt
Resolving lila.science (lila.science)... 20.83.252.133
Connecting to lila.science (lila.science)|20.83.252.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 281461925 (268M)
Saving to: ‘/content/md_v5a.0.0.pt’

/content/md_v5a.0.0 100%[===================>] 268.42M  35.9MB/s    in 8.1s    

2023-07-11 01:17:57 (33.2 MB/s) - ‘/content/md_v5a.0.0.pt’ saved [281461925/281461925]

--2023-07-11 01:17:57--  https://lila.science/public/md_rebuild/md_v5b.0.0_rebuild_pt-1.12_zerolr.pt
Resolving lila.science (lila.science)... 20.83.252.133
Connecting to lila.science (lila.science)|20.83.252.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 281461925 (268M)
Saving to: ‘/content/md_v5b.0.0.pt’

/content/md_v5b.0.0 100%[===================>] 268.42M  38.2MB/s    in 7.6s    

2023-07-11 01:18:05 (35.5 MB/s) - ‘/content/md_v5b.0.0.pt’ saved [2814

In [5]:
!rm -rf /content/MegaDetector
!rm -rf /content/yolov5
!git clone https://github.com/agentmorris/MegaDetector /content/MegaDetector
!git clone https://github.com/ultralytics/yolov5 /content/yolov5

Cloning into '/content/MegaDetector'...
remote: Enumerating objects: 16389, done.
remote: Counting objects: 100% (1235/1235), done.
remote: Compressing objects: 100% (586/586), done.
remote: Total 16389 (delta 711), reused 1147 (delta 647), pack-reused 15154
Receiving objects: 100% (16389/16389), 181.55 MiB | 28.24 MiB/s, done.
Resolving deltas: 100% (9907/9907), done.
Updating files: 100% (865/865), done.
Cloning into '/content/yolov5'...
remote: Enumerating objects: 15814, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 15814 (delta 9), reused 23 (delta 2), pack-reused 15768
Receiving objects: 100% (15814/15814), 14.64 MiB | 19.39 MiB/s, done.
Resolving deltas: 100% (10821/10821), done.


### Set `PYTHONPATH` to include `MegaDetector` and `yolov5`

Add cloned git folders to the `PYTHONPATH` environment variable so that we can import their modules from any working directory.


In [6]:
import os
os.environ['PYTHONPATH'] += ":/content/MegaDetector"
os.environ['PYTHONPATH'] += ":/content/yolov5"

# Run the detection script

There are actually two variants of MegaDetector v5, called "v5a" and "v5b".  By default this notebook runs MDv5a; change "md_v5a.0.0.pt" to "md_v5b.0.0.pt" to run MDv5b instead.

Both run at the same speed; if you are in a Colab session with a GPU accelerator, you should be able to process around four images per second.

In [7]:
!python /content/MegaDetector/detection/run_detector_batch.py md_v5a.0.0.pt "$images_dir" "$output_file_path" --recursive --output_relative_filenames --quiet

21 image files found in the input directory
PyTorch reports 1 available CUDA devices
GPU available: True
Using PyTorch version 2.0.1+cu118
Fusing layers... 
Model summary: 574 layers, 139990096 parameters, 0 gradients
Sending model to GPU
Loaded model in 9.73 seconds
Loaded model in 9.73 seconds
100% 21/21 [00:07<00:00,  2.69it/s]
Finished inference for 21 images in 19.12 seconds (1.10 images per second)
Output file saved at /content/drive/MyDrive/Camaras Aspen/sample.json
Done!


In [8]:
import json
import os
import shutil

def save_images_by_bbox_confidence(json_file, images_dir, confidence_threshold=0.2):
    # Create folders if they don't exist
    detections_dir = os.path.join(images_dir,"detections")
    no_detections_dir = os.path.join(images_dir,"no_detections")
    os.makedirs(detections_dir, exist_ok=True)
    os.makedirs(no_detections_dir, exist_ok=True)

    with open(json_file, 'r') as file:
        data = json.load(file)

        for image_data in data['images']:
            image_path = os.path.basename(image_data['file'])

            # Check if any bbox has confidence over the threshold
            has_bbox_with_high_confidence = False
            for bbox in image_data['detections']:
                if bbox['category'] != '1': continue
                if bbox['conf'] > confidence_threshold:
                    has_bbox_with_high_confidence = True
                    break

            if has_bbox_with_high_confidence:
                # Move image to folder with bbox
                os.rename(os.path.join(images_dir, image_path), os.path.join(detections_dir, image_path))
            else:
                # Move image to folder without bbox
                os.rename(os.path.join(images_dir, image_path), os.path.join(no_detections_dir, image_path))

# Usage example
save_images_by_bbox_confidence(output_file_path, images_dir, confidence_threshold)
